In [1]:
import requests
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from time import sleep

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"}
url = 'https://www.vivino.com/'

# Get Cache key to get country codes and type of wines
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
script = soup.find('script', text = re.compile('var vivinoCacheKey'))
vivinoCacheKey = str(script).split('vivinoCacheKey = ')[-1].split(';')[0].replace("'",'').strip()

# Get countries
api_url = 'https://www.vivino.com/api/countries'
payload = {
    'cache_key':vivinoCacheKey}
countrys = ['fr', 'it', 'us', 'de', 'es', 'au', 'at', 'cl', 'pt', 'ar']


rows = []
# Iterate through countries and wine types
api_url = 'https://www.vivino.com/api/explore/explore'
for country in countrys:
    payload = {
    "country_code": country.upper(),
    "currency_code":"USD",
    "grape_filter":"varietal",
    "min_rating":1,
    "min_ratings_count":500, 
    "order_by":"ratings_count",
    "order":"desc",
    "page": 1,
    "price_range_max": 50,
    "price_range_min": 10,
    "wine_type_id": 1}

    try:
        jsonData = requests.get(api_url, params=payload, headers=headers).json()
        total_pages = math.ceil(jsonData['explore_vintage']['records_matched'] / 100)
        #print('%s' %(country['code'].upper()))
        
        for page in range(1,total_pages+1):
            if page != 1:   
                payload.update({'page':page})
            jsonData = requests.get(api_url, params=payload, headers=headers).json()
            for each in jsonData['explore_vintage']['records']:
                name = f'{each["vintage"]["wine"]["name"]} {each["vintage"]["year"]}'
                year = each["vintage"]["year"]
                id = each["vintage"]["wine"]["id"]
                rating =  each['vintage']['statistics']['ratings_average']
                price = each['price']['amount']
                winery = each["vintage"]["wine"]["winery"]["name"]
                winery_id = each["vintage"]["wine"]["winery"]["id"]
                ratings_count = each["vintage"]["statistics"]["ratings_count"]
                country_name =  country
                number_of_flavors_in_wine = int(len(each['vintage']['wine']['taste']['flavor']))
                flavors_in_wine = []
                for flavor in range(number_of_flavors_in_wine):
                    flavors_in_wine.append(each['vintage']['wine']['taste']['flavor'][flavor]) #flavor(s)
                # grapes_in_wine = []
                # for grape in range(number_of_grapes_in_wine):
                #     grapes_in_wine.append(each["vintage"]["wine"]["style"]["grapes"][grape]["name"]) #Grape(s)

                
                
                row = {'name':name, "year":year, "wine ID":id, 'rating':rating, 'price':price, 'winery': winery, 'winery_id': winery_id, 'ratings_count': ratings_count, 'country_name': country_name,\
                        'number_of_flavors': number_of_flavors_in_wine, 'flavors_in_wine': flavors_in_wine}



                rows.append(row)
            print('Aquired page: %s - %s ' %(country, page))
    except:
        continue

df = pd.DataFrame(rows)


Aquired page: fr - 1 
Aquired page: fr - 2 
Aquired page: fr - 3 
Aquired page: fr - 4 
Aquired page: fr - 5 
Aquired page: fr - 6 
Aquired page: fr - 7 
Aquired page: fr - 8 
Aquired page: fr - 9 
Aquired page: fr - 10 
Aquired page: fr - 11 
Aquired page: fr - 12 
Aquired page: fr - 13 
Aquired page: fr - 14 
Aquired page: fr - 15 
Aquired page: fr - 16 
Aquired page: fr - 17 
Aquired page: fr - 18 
Aquired page: fr - 19 
Aquired page: fr - 20 
Aquired page: fr - 21 
Aquired page: fr - 22 
Aquired page: fr - 23 
Aquired page: fr - 24 
Aquired page: fr - 25 
Aquired page: fr - 26 
Aquired page: fr - 27 
Aquired page: fr - 28 
Aquired page: fr - 29 
Aquired page: fr - 30 
Aquired page: fr - 31 
Aquired page: fr - 32 
Aquired page: fr - 33 
Aquired page: fr - 34 
Aquired page: fr - 35 
Aquired page: fr - 36 
Aquired page: fr - 37 
Aquired page: fr - 38 
Aquired page: fr - 39 
Aquired page: fr - 40 
Aquired page: fr - 41 
Aquired page: fr - 42 
Aquired page: fr - 43 
Aquired page: fr - 4

In [4]:
df = df[df.ratings_count>100]
df = df.drop_duplicates(subset = ['year', 'wine ID'] , keep = 'first')
df.to_csv("asset/wines20221010.csv", index=False)
df

In [6]:
def get_wine_data(wine_id, year, page):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    }

    api_url = "https://www.vivino.com/api/wines/{id}/reviews?per_page=50&year={year}&page={page}"  # <-- increased the number of reviews to 9999

    data = requests.get(
        api_url.format(id=wine_id, year=year, page=page), headers=headers
    ).json()

    return data

In [7]:
ratings=[]
for _, row in df.iterrows():
    page = 1
    while True:
        print(
            f'Getting info about wine {row["wine ID"]}-{row["year"]} Page {page}'
            )
        try:
            d = get_wine_data(row["wine ID"], row["year"], page)
        except:
            sleep(10)
            continue

        try:
            for r in d["reviews"]:
                if r["language"] != "en": # <-- get only english reviews
                    continue

                ratings.append(
                        [
                            row["wine ID"],
                            r["rating"],
                            r["note"],
                            r["created_at"],
                            r['vintage']['wine']['region']['country']['name']
                        ]
                    )
        except:
            break


        if page == 2:
            break

        page += 1


    df_reviews = pd.DataFrame(
        ratings, columns=["wine ID", "User Rating", "Review", "CreatedAt", "country"]
    )

    df_out = df_reviews.merge(df)

df = df.drop_duplicates(subset = ['Review'] , keep = 'first')
df_out.to_pickle("asset/reviews20221010.pkl")

Getting info about wine 1128385-N.V. Page 1
Getting info about wine 1128385-N.V. Page 2
Getting info about wine 1122095-N.V. Page 1
Getting info about wine 1122095-N.V. Page 2
Getting info about wine 21646-N.V. Page 1
Getting info about wine 21646-N.V. Page 2
Getting info about wine 18929-N.V. Page 1
Getting info about wine 18929-N.V. Page 2
Getting info about wine 1129619-N.V. Page 1
Getting info about wine 1129619-N.V. Page 2
Getting info about wine 5552281-N.V. Page 1
Getting info about wine 5552281-N.V. Page 2
Getting info about wine 79160-N.V. Page 1
Getting info about wine 79160-N.V. Page 2
Getting info about wine 2459848-N.V. Page 1
Getting info about wine 2459848-N.V. Page 2
Getting info about wine 1884610-N.V. Page 1
Getting info about wine 1884610-N.V. Page 2
Getting info about wine 1141069-N.V. Page 1
Getting info about wine 1141069-N.V. Page 2
Getting info about wine 1122092-N.V. Page 1
Getting info about wine 1122092-N.V. Page 2
Getting info about wine 1178210-N.V. Page 1
